Loading Data 
1. Air Quality Data

In [1]:
!pip install fiona
!pip install geopandas
!pip install folium
!pip install sodapy -q
!pip install adjustText
!pip install plotly --upgrade
!pip install contextily
!pip install mapclassify
!pip install chart-studio
!pip install pygeos
!sudo apt install libspatialindex-dev
!pip install rtree

     |████████████████████████████████| 15.4 MB 124 kB/s 
     |████████████████████████████████| 1.0 MB 4.9 MB/s 
     |████████████████████████████████| 6.3 MB 40.1 MB/s 
  Created wheel for adjustText: filename=adjustText-0.7.3-py3-none-any.whl size=7094 sha256=8375036b156fdcc81430faf82c08e3a0a249d191029bfcb8a3c15d82dfe262d2
  Stored in directory: /root/.cache/pip/wheels/2f/98/32/afbf902d8f040fadfdf0a44357e4ab750afe165d873bf5893d
Successfully built adjustText
     |████████████████████████████████| 25.3 MB 53.9 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 19.3 MB 7.1 MB/s 
     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 2.1 MB 5.0 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  

In [2]:
# To ignore unimporant system warnings
import warnings
warnings.filterwarnings("ignore")

# We will use Pandas, Numpy, and Matplotlib which is a package for visualization with Python
import pandas as pd
import numpy as np

# Load a required package 
# This is a library for accessing and parsing data through URLs
from urllib.parse import urlencode
import urllib.request, json 
from bs4 import BeautifulSoup # for web scraping
import fiona
import plotly.express as px
import geopandas as gpd
import adjustText as aT
import seaborn as sns # visualization styling package
import plotly.graph_objects as go
import urllib, json
import chart_studio.plotly as py
import chart_studio.tools as tls
import os
from sodapy import Socrata
from IPython.display import Image, display
from shapely.geometry import Polygon
import time
from shapely.geometry import shape
import contextily as ctx
from google.colab import files

# A magic functin that renders the figure in a notebook 
%matplotlib inline 

from matplotlib import pyplot as plt

Getting NYC Air Quality Data from NYC Open Data Portal : https://data.cityofnewyork.us/Environment/Air-Quality/c3uy-2p5r

In [3]:
API = 'https://data.cityofnewyork.us/resource/c3uy-2p5r.csv?'

In [4]:
query = {'$select':'*',
         '$where': 'start_date > "2014-12-31"',
         '$order': 'unique_id',
         '$limit': 14500} #change to 414500 later

aq_df = pd.read_csv(API + urlencode(query))
aq_df.head()

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value,message
0,179718,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,504,South Beach - Tottenville,2015,2015-01-01T00:00:00.000,2.0,NaN
1,179719,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,503,Willowbrook,2015,2015-01-01T00:00:00.000,2.1,NaN
2,179720,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,501,Port Richmond,2015,2015-01-01T00:00:00.000,2.8,NaN
3,179721,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,502,Stapleton - St. George,2015,2015-01-01T00:00:00.000,4.6,NaN
4,179722,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,410,Rockaways,2015,2015-01-01T00:00:00.000,6.1,NaN


In [5]:
aq_df.dtypes

unique_id           int64
indicator_id        int64
name               object
measure            object
measure_info       object
geo_type_name      object
geo_join_id         int64
geo_place_name     object
time_period        object
start_date         object
data_value        float64
message           float64
dtype: object

In [6]:
aq_df['Date'] = pd.to_datetime(aq_df['start_date']).dt.date
aq_df['Time'] = pd.to_datetime(aq_df['start_date']).dt.time

In [7]:
print(aq_df.shape[0])
aq_df.dropna()
print(aq_df.shape[0])

4752
4752


In [8]:
aq_df.head()

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value,message,Date,Time
0,179718,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,504,South Beach - Tottenville,2015,2015-01-01T00:00:00.000,2.0,NaN,2015-01-01,00:00:00
1,179719,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,503,Willowbrook,2015,2015-01-01T00:00:00.000,2.1,NaN,2015-01-01,00:00:00
2,179720,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,501,Port Richmond,2015,2015-01-01T00:00:00.000,2.8,NaN,2015-01-01,00:00:00
3,179721,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,502,Stapleton - St. George,2015,2015-01-01T00:00:00.000,4.6,NaN,2015-01-01,00:00:00
4,179722,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,410,Rockaways,2015,2015-01-01T00:00:00.000,6.1,NaN,2015-01-01,00:00:00


In [9]:
aq_df.dtypes

unique_id           int64
indicator_id        int64
name               object
measure            object
measure_info       object
geo_type_name      object
geo_join_id         int64
geo_place_name     object
time_period        object
start_date         object
data_value        float64
message           float64
Date               object
Time               object
dtype: object

In [10]:
aq_df = aq_df.rename(columns={"geo_join_id":"boro_cd"})

In [11]:
!git clone https://github.com/kirthi-b/QoL_NYC_Children/

Cloning into 'QoL_NYC_Children'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (281/281), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 281 (delta 122), reused 208 (delta 68), pack-reused 0
Receiving objects: 100% (281/281), 23.58 MiB | 11.07 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [12]:
gdf = gpd.read_file("/content/QoL_NYC_Children/Environmental/Data/Air_Quality/Community Districts.zip")

In [13]:
gdf.head()

,boro_cd,shape_area,shape_leng,geometry
0,206.0,4.266431e+07,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843..."
1,404.0,6.573966e+07,37018.373718,"POLYGON ((-73.84751 40.73901, -73.84801 40.738..."
2,203.0,4.479687e+07,33500.069106,"POLYGON ((-73.88072 40.83752, -73.88074 40.837..."
3,304.0,5.666261e+07,37007.806599,"POLYGON ((-73.89647 40.68234, -73.89653 40.682..."
4,205.0,3.831698e+07,29443.048128,"POLYGON ((-73.89138 40.86170, -73.89142 40.861..."


In [14]:
gdf.dtypes

boro_cd        float64
shape_area     float64
shape_leng     float64
geometry      geometry
dtype: object

In [15]:
gdf = gdf.astype({"boro_cd":'int'})
gdf.head()

,boro_cd,shape_area,shape_leng,geometry
0,206,4.266431e+07,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843..."
1,404,6.573966e+07,37018.373718,"POLYGON ((-73.84751 40.73901, -73.84801 40.738..."
2,203,4.479687e+07,33500.069106,"POLYGON ((-73.88072 40.83752, -73.88074 40.837..."
3,304,5.666261e+07,37007.806599,"POLYGON ((-73.89647 40.68234, -73.89653 40.682..."
4,205,3.831698e+07,29443.048128,"POLYGON ((-73.89138 40.86170, -73.89142 40.861..."


In [16]:
gdf_join = gdf.merge(aq_df, on="boro_cd", how='left')
gdf_join.drop_duplicates()
gdf_join.drop(columns=['message','start_date'])
gdf_join.dropna()
gdf_join.head()

,boro_cd,shape_area,shape_leng,geometry,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_place_name,time_period,start_date,data_value,message,Date,Time
0,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179744.0,642.0,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,34.4,NaN,2015-01-01,00:00:00
1,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179792.0,640.0,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,1.1,NaN,2015-01-01,00:00:00
2,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179840.0,641.0,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,0.2,NaN,2015-01-01,00:00:00
3,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325103.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,Borough Park,2016,2016-01-01T00:00:00.000,0.6,NaN,2016-01-01,00:00:00
4,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325151.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,CD,Belmont and East Tremont (CD6),2016,2016-01-01T00:00:00.000,3.8,NaN,2016-01-01,00:00:00


In [17]:
print (gdf_join.crs)

epsg:4326


In [18]:
gdf_join["center"] = gdf_join["geometry"].centroid
gdf_join = gdf_join.rename(columns={"geometry":"geopolygon"})
gdf_join = gdf_join.set_geometry("center")
gdf_join = gdf_join.rename(columns={"center":"geometry"})
gdf_join.head()

,boro_cd,shape_area,shape_leng,geopolygon,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_place_name,time_period,start_date,data_value,message,Date,Time,geometry
0,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179744.0,642.0,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,34.4,NaN,2015-01-01,00:00:00,POINT (-73.88753 40.84960)
1,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179792.0,640.0,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,1.1,NaN,2015-01-01,00:00:00,POINT (-73.88753 40.84960)
2,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179840.0,641.0,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,0.2,NaN,2015-01-01,00:00:00,POINT (-73.88753 40.84960)
3,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325103.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,Borough Park,2016,2016-01-01T00:00:00.000,0.6,NaN,2016-01-01,00:00:00,POINT (-73.88753 40.84960)
4,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325151.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,CD,Belmont and East Tremont (CD6),2016,2016-01-01T00:00:00.000,3.8,NaN,2016-01-01,00:00:00,POINT (-73.88753 40.84960)


In [19]:
puma_gdf = gpd.read_file("/content/QoL_NYC_Children/Access/Data/PUMA", driver = "shapefile")
puma_gdf.head()

,PUMA,Shape_Leng,Shape_Area,geometry
0,3701,53227.113608,9.792852e+07,"POLYGON ((1015289.353 261467.596, 1015334.054 ..."
1,3702,106167.592328,1.889937e+08,"POLYGON ((1021632.336 267934.439, 1022108.577 ..."
2,3703,305269.138803,2.676436e+08,"MULTIPOLYGON (((1042822.450 243439.814, 104255..."
3,3704,47970.203023,1.062169e+08,"POLYGON ((1025507.170 246350.149, 1025418.562 ..."
4,3705,68697.599919,1.224837e+08,"POLYGON ((1012111.566 240863.078, 1012088.246 ..."


In [20]:
puma_gdf.shape[0]

55

In [21]:
print (puma_gdf.crs)

epsg:2263


In [22]:
puma_gdf = puma_gdf.to_crs(4326)

In [23]:
gdf_aq_puma = gpd.sjoin(gdf_join, puma_gdf, how='left')
gdf_aq_puma.head()

,boro_cd,shape_area,shape_leng,geopolygon,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_place_name,time_period,start_date,data_value,message,Date,Time,geometry,index_right,PUMA,Shape_Leng,Shape_Area
0,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179744.0,642.0,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,34.4,NaN,2015-01-01,00:00:00,POINT (-73.88753 40.84960),4.0,3705,68697.599919,1.224837e+08
1,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179792.0,640.0,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,1.1,NaN,2015-01-01,00:00:00,POINT (-73.88753 40.84960),4.0,3705,68697.599919,1.224837e+08
2,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179840.0,641.0,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,0.2,NaN,2015-01-01,00:00:00,POINT (-73.88753 40.84960),4.0,3705,68697.599919,1.224837e+08
3,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325103.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,Borough Park,2016,2016-01-01T00:00:00.000,0.6,NaN,2016-01-01,00:00:00,POINT (-73.88753 40.84960),4.0,3705,68697.599919,1.224837e+08
4,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325151.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,CD,Belmont and East Tremont (CD6),2016,2016-01-01T00:00:00.000,3.8,NaN,2016-01-01,00:00:00,POINT (-73.88753 40.84960),4.0,3705,68697.599919,1.224837e+08


In [24]:
gdf_aq_puma = gdf_aq_puma.groupby('PUMA').data_value.mean()
gdf_aq_puma = gdf_aq_puma.to_frame().reset_index()
gdf_aq_puma.head()

,PUMA,data_value
0,3701,16.491276
1,3702,14.841000
2,3703,15.336200
3,3704,19.213100
4,3705,17.908700


In [25]:
aq_gdf=puma_gdf.copy()
aq_gdf=aq_gdf.join(gdf_aq_puma.set_index('PUMA'), on='PUMA')
aq_gdf.dropna()
aq_gdf.head()

,PUMA,Shape_Leng,Shape_Area,geometry,data_value
0,3701,53227.113608,9.792852e+07,"POLYGON ((-73.88775 40.88429, -73.88759 40.884...",16.491276
1,3702,106167.592328,1.889937e+08,"POLYGON ((-73.86477 40.90201, -73.86305 40.901...",14.841000
2,3703,305269.138803,2.676436e+08,"MULTIPOLYGON (((-73.78833 40.83467, -73.78931 ...",15.336200
3,3704,47970.203023,1.062169e+08,"POLYGON ((-73.85089 40.84275, -73.85121 40.842...",19.213100
4,3705,68697.599919,1.224837e+08,"POLYGON ((-73.89933 40.82775, -73.89941 40.827...",17.908700


In [26]:
#Un-comment the following lines to export 

#aq_gdf.to_csv('aq_gdf.csv')
#files.download('aq_gdf.csv')

**Water Quality 2015 - 2019**

In [27]:
API_2 = 'https://data.cityofnewyork.us/resource/bkwf-xfky.csv?'


In [28]:
query_2 = {'$select':'*',
      '$where': 'sample_date < "2020-01-01"',
      '$order': 'sample_number',
      '$limit': 90000} #change to 100k later

wq_df = pd.read_csv(API_2 + urlencode(query_2))
wq_df.head()

,sample_number,sample_date,sample_time,sample_site,sample_class,residual_free_chlorine_mg_l,turbidity_ntu,fluoride_mg_l,coliform_quanti_tray_mpn_100ml,e_coli_quanti_tray_mpn_100ml
0,201500023,2015-01-01T00:00:00.000,12:19,1S07,Operational,0.58,0.96,0.79,<1,<1
1,201500024,2015-01-01T00:00:00.000,11:15,1S04,Operational,0.71,0.94,0.80,<1,<1
2,201500025,2015-01-01T00:00:00.000,10:09,1S03A,Operational,0.79,0.93,0.79,<1,<1
3,201500026,2015-01-01T00:00:00.000,10:41,1S03B,Operational,0.77,0.93,0.80,<1,<1
4,201500027,2015-01-01T00:00:00.000,09:38,11550,Compliance,0.74,0.95,NaN,<1,<1


In [29]:
wq_df.dtypes

sample_number                       int64
sample_date                        object
sample_time                        object
sample_site                        object
sample_class                       object
residual_free_chlorine_mg_l       float64
turbidity_ntu                      object
fluoride_mg_l                      object
coliform_quanti_tray_mpn_100ml     object
e_coli_quanti_tray_mpn_100ml       object
dtype: object

In [30]:
wq_df['Date'] = pd.to_datetime(wq_df['sample_date']).dt.date
wq_df['Time'] = pd.to_datetime(wq_df['sample_date']).dt.time

In [31]:
print(wq_df.shape[0])
wq_df.dropna()
print(wq_df.shape[0])

79335
79335


In [32]:
wq_df.head()

,sample_number,sample_date,sample_time,sample_site,sample_class,residual_free_chlorine_mg_l,turbidity_ntu,fluoride_mg_l,coliform_quanti_tray_mpn_100ml,e_coli_quanti_tray_mpn_100ml,Date,Time
0,201500023,2015-01-01T00:00:00.000,12:19,1S07,Operational,0.58,0.96,0.79,<1,<1,2015-01-01,00:00:00
1,201500024,2015-01-01T00:00:00.000,11:15,1S04,Operational,0.71,0.94,0.80,<1,<1,2015-01-01,00:00:00
2,201500025,2015-01-01T00:00:00.000,10:09,1S03A,Operational,0.79,0.93,0.79,<1,<1,2015-01-01,00:00:00
3,201500026,2015-01-01T00:00:00.000,10:41,1S03B,Operational,0.77,0.93,0.80,<1,<1,2015-01-01,00:00:00
4,201500027,2015-01-01T00:00:00.000,09:38,11550,Compliance,0.74,0.95,NaN,<1,<1,2015-01-01,00:00:00


In [37]:
wsite_df = pd.read_excel("/content/QoL_NYC_Children/Environmental/Data/Water_Quality/OpenData_Distribution_Water_Quality_Sampling_Sites_Updated_2021-0618.xlsx")
wsite_df.head()

,Sample Site,Sample Station (SS) - Location Description,X - Coordinate,Y - Coordinate
0,1S03,SS - Shaft 3 of City Tunnel No.1 - E/S Goulden...,1024950,264277
1,1S04,SS - Shaft 4 of City Tunnel No.1 - IFO 2780 Re...,1012568,256577
2,1S03A,SS - Shaft 3A of City Tunnel No.2 - S/S E 233r...,1024721,264392
3,1S07,SS - Shaft 7 of City Tunnel No.1 - NE/S W 167t...,1004013,245233
4,1S03B,SS - Shaft 3B of City Tunnel No.3 - W/S Jerome...,1014949,260688


In [38]:
wsite_df = wsite_df.rename(columns={"Sample Site":"sample_site"})

In [39]:
wq_df_join = wsite_df.merge(wq_df, on="sample_site", how='left')

In [41]:
wq_df_join.drop_duplicates()
wq_df_join.drop(columns=['sample_class','Time','sample_time'])
wq_df_join.dropna()
wq_df_join.head()

,sample_site,Sample Station (SS) - Location Description,X - Coordinate,Y - Coordinate,sample_number,sample_date,sample_time,sample_class,residual_free_chlorine_mg_l,turbidity_ntu,fluoride_mg_l,coliform_quanti_tray_mpn_100ml,e_coli_quanti_tray_mpn_100ml,Date,Time
0,1S03,SS - Shaft 3 of City Tunnel No.1 - E/S Goulden...,1024950,264277,201668253.0,2016-12-10T00:00:00.000,10:12,Operational,0.68,0.69,0.72,<1,<1,2016-12-10,00:00:00
1,1S03,SS - Shaft 3 of City Tunnel No.1 - E/S Goulden...,1024950,264277,201668328.0,2016-12-11T00:00:00.000,09:11,Operational,0.69,0.61,0.72,<1,<1,2016-12-11,00:00:00
2,1S03,SS - Shaft 3 of City Tunnel No.1 - E/S Goulden...,1024950,264277,201668403.0,2016-12-12T00:00:00.000,11:47,Operational,0.78,0.6,0.70,<1,<1,2016-12-12,00:00:00
3,1S03,SS - Shaft 3 of City Tunnel No.1 - E/S Goulden...,1024950,264277,201668493.0,2016-12-13T00:00:00.000,10:53,Operational,0.75,0.54,0.70,<1,<1,2016-12-13,00:00:00
4,1S03,SS - Shaft 3 of City Tunnel No.1 - E/S Goulden...,1024950,264277,201668608.0,2016-12-14T00:00:00.000,10:54,Operational,0.79,0.6,0.74,<1,<1,2016-12-14,00:00:00


**2.NYC Parks Data**

In [ ]:
gdf_parks = gpd.read_file("/content/QoL_NYC_Children/Environmental/Data/Parks/ARCHIVED - Parks Properties.zip")

In [ ]:
gdf_parks.head()

In [ ]:
gdf_parks.dtypes

In [ ]:
gdf_parks.drop_duplicates()
gdf_parks.drop(columns=['acquisitio','class','waterfront','us_congres','url','retired','pip_ratabl','permitpare','permitdist','permit','parentid','nys_senate','nys_assemb','global_id','mapped'])

In [ ]:
gdf_parks.dropna()
print(gdf_parks.shape[0])

In [ ]:
print (gdf_parks.crs)
print (gdf_parks.crs.name)
print (gdf_parks.crs.datum)

In [ ]:
gdf_parks["center"] = gdf_parks["geometry"].centroid
gdf_parks.head()

In [ ]:
#gdf_parks.to_csv('NYCParks.csv', index = False)
#from google.colab import files
#files.download("NYCParks.csv")